In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Skripsi/Dataset/data_bersih_GPR.csv")

df['DATE'] = pd.to_datetime(df['DATE'])

# SORT BERDASARKAN WAKTU
df = df.sort_values('DATE').reset_index(drop=True)

print("Jumlah data awal:", len(df))

Jumlah data awal: 10569


In [3]:
df['silver_return'] = df['SILVER_PRICE'].pct_change()
df['gold_return']   = df['GOLD_PRICE'].pct_change()

# Hapus hanya baris pertama (karena pct_change)
df = df.iloc[1:].reset_index(drop=True)

print("Jumlah data setelah return:", len(df))

Jumlah data setelah return: 10568


/tmp/ipython-input-1952581561.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['silver_return'] = df['SILVER_PRICE'].pct_change()


In [4]:
features = ['gold_return','silver_return','GPRD']
data = df[features].values

In [5]:
train_size = int(len(data) * 0.8)

train_data = data[:train_size]
test_data  = data[train_size:]

In [6]:
scaler = MinMaxScaler()

train_scaled = scaler.fit_transform(train_data)
test_scaled  = scaler.transform(test_data)

data_scaled = np.vstack((train_scaled, test_scaled))

In [7]:
def create_dataset(dataset, time_step):
    X, y = [], []
    for i in range(len(dataset)-time_step-1):
        X.append(dataset[i:(i+time_step), :])
        y.append(dataset[i+time_step, 1])
    return np.array(X), np.array(y)

time_step = 20

X, y = create_dataset(data_scaled, time_step)

print("Shape X:", X.shape)
print("Shape y:", y.shape)

Shape X: (10547, 20, 3)
Shape y: (10547,)


In [8]:
train_size_window = int(len(X) * 0.8)

X_train = X[:train_size_window]
X_test  = X[train_size_window:]

y_train = y[:train_size_window]
y_test  = y[train_size_window:]

print("Final X_train shape:", X_train.shape)

Final X_train shape: (8437, 20, 3)


#LSTM

In [9]:
model_lstm = Sequential([
    Input(shape=(time_step, 3)),
    LSTM(64),
    Dropout(0.2),
    Dense(1)
])

model_lstm.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history_lstm = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0182 - val_loss: 0.0026
Epoch 2/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0043 - val_loss: 0.0027
Epoch 3/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0041 - val_loss: 0.0032
Epoch 4/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0039 - val_loss: 0.0027
Epoch 5/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.0036 - val_loss: 0.0025
Epoch 6/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0034 - val_loss: 0.0025
Epoch 7/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0033 - val_loss: 0.0025
Epoch 8/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0034 - val_loss: 0.0025
Epoch 9/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0031 - val_loss: 0.0025
Epoch 10/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0030 - val_loss: 0.0025
Epoch 11/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0030 - val_loss: 0.0024
Epoch 12/100
264/264 ━━━━━━━━━━━━━━

#GRU

In [10]:
model_gru = Sequential([
    Input(shape=(time_step, 3)),
    GRU(64),
    Dropout(0.2),
    Dense(1)
])

model_gru.compile(optimizer='adam', loss='mse')

history_gru = model_gru.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 0.0651 - val_loss: 0.0026
Epoch 2/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0048 - val_loss: 0.0026
Epoch 3/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 0.0039 - val_loss: 0.0025
Epoch 4/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0040 - val_loss: 0.0026
Epoch 5/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0037 - val_loss: 0.0025
Epoch 6/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0039 - val_loss: 0.0025
Epoch 7/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.0037 - val_loss: 0.0025
Epoch 8/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0035 - val_loss: 0.0026
Epoch 9/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0033 - val_loss: 0.0024
Epoch 10/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0035 - val_loss: 0.0024
Epoch 11/100
264/264 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0030 - val_loss: 0.0026
Epoch 12/100
264/264 ━━━━━━━━━

#Eval

In [11]:
pred_lstm = model_lstm.predict(X_test)
pred_gru  = model_gru.predict(X_test)

66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [12]:
def inverse_return(pred_scaled):
    dummy = np.zeros((len(pred_scaled), 3))
    dummy[:,1] = pred_scaled[:,0]
    inv = scaler.inverse_transform(dummy)
    return inv[:,1]

pred_lstm_inv = inverse_return(pred_lstm)
pred_gru_inv  = inverse_return(pred_gru)
y_test_inv    = inverse_return(y_test.reshape(-1,1))

In [13]:
rmse_lstm = np.sqrt(mean_squared_error(y_test_inv, pred_lstm_inv))
rmse_gru  = np.sqrt(mean_squared_error(y_test_inv, pred_gru_inv))

print("RMSE LSTM:", rmse_lstm)
print("RMSE GRU:", rmse_gru)

RMSE LSTM: 0.01738706880351662
RMSE GRU: 0.017419129761058215


In [14]:
best_rmse = 999
best_alpha = 0

for alpha in np.arange(0,1.01,0.01):
    pred = alpha*pred_lstm_inv + (1-alpha)*pred_gru_inv
    rmse = np.sqrt(mean_squared_error(y_test_inv, pred))

    if rmse < best_rmse:
        best_rmse = rmse
        best_alpha = alpha

print("Best alpha:", best_alpha)
print("Best RMSE:", best_rmse)

Best alpha: 1.0
Best RMSE: 0.01738706880351662
